# Tutorial: Downloading and preprocessing data
---

In this notebok, we look at how to download and preprocess data.

In [2]:
from ggdata import download

API_name = 'WB'

config = {
    'GGI_code': 'EE2',
    'params': {'indicator': 'EG.FEC.RNEW.ZS'}
}

data = download(API_name, config, raw=False)

PreProcessing: DONE


## Quick Start

1. Choose an API
2. Define a config
3. Get the data 

#### 1. Choose the  API

The are 3 options: 'WB' for world bank, 'SDG' for UNSTAT SDG, 'CW' for climate watch.

#### 2. Define a config
This part is the "key" part as different APIs have differents standards and code, explore their documentations to find the codes you need. Then, define a config as the following:
A dictionnary with 2 keys:
- GGI_code: The name you want to give your variable
- params: A dictionnary formatted with respect to the API documentation

Here are a few examples

In [3]:
WB_config_sample = {
    'GGI_code': 'EE2',
    'params': {'indicator': 'EG.FEC.RNEW.ZS'}
}

SDG_config_sample = {
    'GGI_code': 'SE2.3',
    'params': {
        'seriesCode': 'EG_ELC_ACCS',
        'dimensions': "[{name:'Location',values:['URBAN']}]"}  # Notice how 'dimensions' is a STRING 
}

CW_config_sample = {
    'GGI_code': 'GE1.0',
    'params': {
        'source_ids[]': 81,  #The parameters in this API are number, refer to the doc to find the one you need ! 
        'sector_ids[]': 957,
        'gas_ids[]': 269}
}

#### 3. Get the data

The download function has 2 additionnal parameters:
- raw to get raw or prefromatted data
- path to directly save the file at the given path

A few prompts will tell you how it is going ! Be patient, the API can take a bit of time to respond.

Using the SDG API

In [4]:
data = download('SDG', SDG_config_sample, raw=True) 

In [5]:
data

{'data': [],
 'metadata': {'URL': 'https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?seriesCode=EG_ELC_ACCS&dimensions=%5B%7Bname%3A%27Location%27%2Cvalues%3A%5B%27URBAN%27%5D%7D%5D&pageSize=1000000000',
  'DownloadDate': '2021-06-03'}}

Using the World bank API to get a raw file 

A raw file is a dictionnary with 2 keys, **data** and **metadata**:
- data is the raw ouput of the API
- metadata is data added by the downloader such as API URL and downloading date

In [6]:
data = download('WB', WB_config_sample, raw=True) 

## Using Submodules

If need be, you can use the module used in the download function for doing more customed processing.

### Getting data

1. Define a Downloader
2. Define a request parameters
3. Get the data
4. Save the data

**IMPORTANT**: The key part of this step is to define the paramters correctly, each API has their own classification and can be checked at the respective documentation, you can look at the one in params/APIs to have some examples.

In [10]:
from ggdata.downloaders.downloader import SDG_Downloader

Downloader = SDG_Downloader('https://unstats.un.org/SDGAPI/v1/sdg/Series/Data')

parameters = {'seriesCode': 'SL_TLF_NEET',
              'dimensions': "[{name:'Sex',values:['BOTHSEX']},{name:'Age',values:['15-24']}]"}

data = Downloader.get_data(parameters)

The data is a dictionnary with 2 keys: 'data' and 'metadata'. Data contains the actual response from the API while metadata contains information about the url and the downloading data.

In [11]:
print(data['metadata'])

{'URL': 'https://unstats.un.org/SDGAPI/v1/sdg/Series/Data?seriesCode=SL_TLF_NEET&dimensions=%5B%7Bname%3A%27Sex%27%2Cvalues%3A%5B%27BOTHSEX%27%5D%7D%2C%7Bname%3A%27Age%27%2Cvalues%3A%5B%2715-24%27%5D%7D%5D&pageSize=1000000000', 'DownloadDate': '2021-06-03'}


In [12]:
print(data['data'][0])

{'goal': ['8'], 'target': ['8.6'], 'indicator': ['8.6.1'], 'series': 'SL_TLF_NEET', 'seriesDescription': 'Proportion of youth not in education, employment or training, by sex and age (%)', 'seriesCount': '5016', 'geoAreaCode': '4', 'geoAreaName': 'Afghanistan', 'timePeriodStart': 2017.0, 'value': '42.0', 'valueType': 'Float', 'time_detail': None, 'timeCoverage': None, 'upperBound': None, 'lowerBound': None, 'basePeriod': None, 'source': 'HIES - Living Condition Survey', 'geoInfoUrl': None, 'footnotes': ['Repository: ILO-STATISTICS - Micro data processing'], 'attributes': {'Nature': 'C', 'Units': 'PERCENT'}, 'dimensions': {'Age': '15-24', 'Sex': 'BOTHSEX', 'Reporting Type': 'G'}}


To save the data directly, you can use the following code: 

In [13]:
data = Downloader.download_data('example.json', parameters) # Save the data as a JSON file

Then, to open the json file use the following command:

In [14]:
import json
with open('example.json') as f:
    data = json.load(f)
print(data['data'][0])

{'goal': ['8'], 'target': ['8.6'], 'indicator': ['8.6.1'], 'series': 'SL_TLF_NEET', 'seriesDescription': 'Proportion of youth not in education, employment or training, by sex and age (%)', 'seriesCount': '5016', 'geoAreaCode': '4', 'geoAreaName': 'Afghanistan', 'timePeriodStart': 2017.0, 'value': '42.0', 'valueType': 'Float', 'time_detail': None, 'timeCoverage': None, 'upperBound': None, 'lowerBound': None, 'basePeriod': None, 'source': 'HIES - Living Condition Survey', 'geoInfoUrl': None, 'footnotes': ['Repository: ILO-STATISTICS - Micro data processing'], 'attributes': {'Nature': 'C', 'Units': 'PERCENT'}, 'dimensions': {'Age': '15-24', 'Sex': 'BOTHSEX', 'Reporting Type': 'G'}}


The data was saved properly !

### Processing the data

1. Define a preprocessor
2. Define additionnal information
2. Process the data


In [15]:
from ggdata.preprocessors.SDG import SDG_Preprocessor

Preprocessor = SDG_Preprocessor('test') # file argument to change (Used to preprocess special cases)

information = {'Variable': 'Test', 'From': 'SDG API'} # Let's you add more information to the dataframe

df = Preprocessor.preprocess(data, information)

df.head()

,Country,ISO,Description,Source,Year,Value,Variable,From,URL,DownloadDate
0,Afghanistan,AFG,"Proportion of youth not in education, employme...",HIES - Living Condition Survey,2017,42.0,Test,SDG API,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-06-03
1,Albania,ALB,"Proportion of youth not in education, employme...",LFS - Labour Force Survey,2007,33.8,Test,SDG API,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-06-03
2,Albania,ALB,"Proportion of youth not in education, employme...",LFS - Labour Force Survey,2008,28.1,Test,SDG API,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-06-03
3,Albania,ALB,"Proportion of youth not in education, employme...",LFS - Labour Force Survey,2009,30.7,Test,SDG API,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-06-03
4,Albania,ALB,"Proportion of youth not in education, employme...",LFS - Labour Force Survey,2010,29.5,Test,SDG API,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-06-03


The data is now preprocessed into a clean and standardized dataframe.